## Данный пример демонстрирует создание нейросети (НС) при помощи интерпретатора специального языка (названия пока нет)

Шаги создания модели:
1. Написать скрипт с архитектурой НС.
2. Создать экземпляр парсера Parser().
3. Создать модели pytorch из скрипта при помощи функции from_str().
4. Извлечь готовый модуль из результата для использования в вычислениях.

In [1]:
import torch
from torchview import draw_graph
from  generator.parser import Parser
import generator.bricks as bricks
import sys
import time
import torch.nn.functional as F

In [2]:
# Тест создания моделей из выражения строки
device = "cuda"

# Примеры
examples = dict(
    s1 = "output={ {@4->relu+@8->relu}^2 }%2->@16->softmax->linear(5);",
    s2 = "output={ {@16->relu+@16->sigmoid}^4 }%8->@16;",
    s3 = "output={ {@64->relu}^64 }%64;",
    s4 = "output = linear(5) -> softmax;",
    s5 = "output = { @5->@20 + @10->@20 + @20 } -> softmax;",
    s6 = """
        y = @64 + @64;          # y - параллельно соединены x и модуль из 64 нейронов
        z = @8 -> y;            # z - x последовательно соединен с y
        w = @8 ^ 4;             # w - 4 слоя по 8 нейронов последовательно соединены
        a = {@16 + @16} % 2;    # a - параллельно соединены два модуля x
        output = z -> w -> a -> {{@8 -> relu + @8 -> relu} ^ 2} % 2 -> @16 -> softmax;
    """
)
# создаем парсер
parser = Parser()

# Отмечаем время старта
start_time = time.time()

# создаем модели примеров
models = [parser.from_str(s) for s in examples.values()]

# Также можем создать модули из json-файла.
# modules = parser.from_json('nntest.json')

# Отмечаем время окончания создания модели
end_time = time.time()

# Результат работы парсера - набор модулей models, в которых храняться модели.
# Чтобы использовать модель - мы можем обратиться к ней по имени соответствующей переменной из скрипта.
model = models[-1]['output'].to(device)

print(f"Время генерации моделей: {end_time-start_time}")

2024-11-09 21:59:43.581 | INFO     | generator.parser:__init__:104 - Инициализация парсера...
Generating LALR tables
2024-11-09 21:59:43.598 | WARNING  | generator.parser:_check_output:243 - Input block not defined in the module.
2024-11-09 21:59:43.636 | WARNING  | generator.parser:_check_output:243 - Input block not defined in the module.
2024-11-09 21:59:45.767 | WARNING  | generator.parser:_check_output:243 - Input block not defined in the module.
2024-11-09 21:59:45.780 | WARNING  | generator.parser:_check_output:243 - Input block not defined in the module.
2024-11-09 21:59:45.786 | WARNING  | generator.parser:_check_output:243 - Input block not defined in the module.
2024-11-09 21:59:45.805 | WARNING  | generator.parser:_check_output:243 - Input block not defined in the module.


Время генерации моделей: 2.217611312866211


In [3]:
# Тестируем работу модели на тестовом тензоре
x = torch.randn(1, 10).to(device)
y = model.to(device)(x)
print(f"Результат:\n{y}")


Результат:
tensor([[0.0502, 0.0742, 0.0598, 0.0819, 0.0695, 0.0447, 0.0784, 0.0672, 0.0502,
         0.0600, 0.0581, 0.0848, 0.0702, 0.0525, 0.0520, 0.0462]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [4]:
# Получим один из элементов модели по идентификатору подмодуля
print(f"Левая подсеть: \n{model.left}")
print(f"Правая компонента левой подсети:\n{model.left.right}")
models

Левая подсеть: 
Composer(
  (left): Composer(
    (left): Composer(
      (left): Composer(
        (left): Composer(
          (left): Linear(
            (linear): Linear(in_features=10, out_features=8, bias=True)
          )
          (right): {Linear(
            (linear): Linear(in_features=8, out_features=64, bias=True)
          )+Linear(
            (linear): Linear(in_features=8, out_features=64, bias=True)
          )}
        )
        (right): Multiplicator(
          (layers): ModuleList(
            (0): Linear(
              (linear): Linear(in_features=64, out_features=8, bias=True)
            )
            (1-3): 3 x Linear(
              (linear): Linear(in_features=8, out_features=8, bias=True)
            )
          )
        )
      )
      (right): Splitter(
        (layers): ModuleList(
          (0-1): 2 x {Linear(
            (linear): Linear(in_features=8, out_features=16, bias=True)
          )+Linear(
            (linear): Linear(in_features=8, out_feature

[{'y': {Linear(
    (linear): Linear(in_features=8, out_features=64, bias=True)
  )+Linear(
    (linear): Linear(in_features=8, out_features=64, bias=True)
  )},
  'z': Composer(
    (left): Linear(
      (linear): Linear(in_features=10, out_features=8, bias=True)
    )
    (right): {Linear(
      (linear): Linear(in_features=8, out_features=64, bias=True)
    )+Linear(
      (linear): Linear(in_features=8, out_features=64, bias=True)
    )}
  ),
  'w': Multiplicator(
    (layers): ModuleList(
      (0): Linear(
        (linear): Linear(in_features=64, out_features=8, bias=True)
      )
      (1-3): 3 x Linear(
        (linear): Linear(in_features=8, out_features=8, bias=True)
      )
    )
  ),
  'a': Splitter(
    (layers): ModuleList(
      (0-1): 2 x {Linear(
        (linear): Linear(in_features=8, out_features=16, bias=True)
      )+Linear(
        (linear): Linear(in_features=8, out_features=16, bias=True)
      )}
    )
  ),
  'output': Composer(
    (left): Composer(
      (lef

In [5]:
# В качестве тестовой модели будем использовать последнюю модель из списка
test_model = models[-1]['output']

In [6]:
# Подсчитаем размер модели
params_count = sum(p.numel() for p in test_model.parameters())
print(f"Параметров: {params_count}, время создания: ", end_time - start_time)
print(f"Размерность выходного тензора: {y.shape}")

Параметров: 3528, время создания:  2.217611312866211
Размерность выходного тензора: torch.Size([1, 16])


In [7]:
# Обратная конвертация модели в выражение
# Это выражение не является полноценным скриптом, т.к. не является выражением присвоения
test_model.expr_str(expand=True)


'{{{{{{linear(8)->{linear(64)+linear(64)}}->linear(8)^4}->{linear(16)+linear(16)}%2}->{{linear(8)->relu}+{linear(8)->relu}}^2%2}->linear(16)}->softmax}'

In [8]:
# Нарисуем диаграмму модели
from generator.visualizers import draw_model


input_size = (1, 10)
pic_path = './pic'
graph_name = 'test'
model_graph = draw_model(test_model, graph_name, pic_path)

print(f"Изображение сохранено в {pic_path}/{graph_name}.png")
model_graph.resize_graph(scale=3)
model_graph.visual_graph.view(graph_name)

Изображение сохранено в ./pic/test.png


'pic/test.pdf'

In [9]:
# Пример того, как можно использовать в вычислениях отдельный подмодуль модели
# Тестовый входной тензор для модели
input_size = (1, 8)
x = torch.randn(input_size).to(device)

# Так как в синтаксисе операции '+' и '->' являются бинарными, 
# построенные из таких выражений подмодули имеют имена left и right
# Извлечем элемент left.right из модели
chunk = test_model.left.right
print(chunk(x))

graph_name='left.right'
model_graph = draw_model(chunk, graph_name, pic_path)

# Более того, мы можем менять структуру, например, операцией decompose()
# Разделим на две части блок chunk
left, right = chunk.decompose()

# Создадим новый модуль как соединение left и  right
new_chunk = bricks.Connector(left, right).to("cuda")
print(new_chunk(x))

graph_name='decomose'
model_graph = draw_model(chunk, graph_name, pic_path)
print()
print(f"Результат разделения и склейки сохранен в {pic_path}/{graph_name}")

tensor([[-2.6101e-01, -8.0922e-04,  1.5166e+00, -9.8678e-02, -6.8415e-02,
         -1.1567e+00,  4.5056e-02,  1.4930e+00,  1.3319e-01,  6.4799e-01,
         -3.5183e-01, -4.0785e-01,  5.2273e-01, -1.2565e+00, -8.5543e-01,
          7.7014e-01]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.8618,  0.9677, -2.3655, -0.4442, -0.5223, -0.5777, -0.5591,  0.2077,
          0.2591,  0.6664,  1.1582,  0.3939, -0.2949, -0.0174, -1.0903,  0.4307,
         -0.4979,  1.2236, -1.1942, -0.4279, -1.1661, -0.9207, -0.3239, -0.0266]],
       device='cuda:0', grad_fn=<CatBackward0>)

Результат разделения и склейки сохранен в ./pic/decomose


In [10]:
print(f"Было:\n{chunk}\n\n")
print(f"После chunk.decompose() и briks.Connector стало:\n{new_chunk}")

Было:
Linear(
  (linear): Linear(in_features=8, out_features=16, bias=True)
)


После chunk.decompose() и briks.Connector стало:
Connector(
  (left): Linear(
    (linear): Linear(in_features=8, out_features=8, bias=True)
  )
  (right): Linear(
    (linear): Linear(in_features=8, out_features=16, bias=True)
  )
)
